Solution4Assignment3

October 9, 2024

• Student: Jason (Yupo) Niu
• Date: 2024-10-09

Scenario 1 - Exponential Smoothing - Import Necessary Libraries & Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
%matplotlib inline
# Read the 'Exponential Smoothing' sheet with numeric headers
df_exp = pd.read_excel('/tmp/Assignment_3_DataSheet.xlsx', sheet_name='Exponential Smoothing', header=2)
# Get the numeric months from the columns, excluding the 'Period' column
months = df_exp.columns[1:]
# Get the actual data from the first row, excluding the 'Period' column
actual_data = df_exp.iloc[0, 1:].values
# Convert the actual data to numeric values
actual_data = pd.to_numeric(actual_data, errors='coerce')
# Create the time_series with numeric indices
time_series = pd.Series(data=actual_data, index=months)

Prepare the Data

In [ ]:
# Drop any NaN values
time_series = time_series.dropna()
# Ensure the index (months) is numeric
time_series.index = pd.to_numeric(time_series.index)

• define custom function for Exponential Smoothing

In [ ]:
def exponential_smoothing(series, alpha):
    """
    Perform exponential smoothing on a time series.
    Parameters:
    - series: Pandas Series containing the time series data.
    - alpha: Smoothing factor (0 < alpha <= 1).
    Returns:
    - Pandas Series containing the smoothed values.
    """
    result = [series.iloc[0]] # Initial forecast equals the first actual value
    for n in range(1, len(series)):
        forecast = alpha * series.iloc[n - 1] + (1 - alpha) * result[n - 1]
        result.append(forecast)
    return pd.Series(result, index=series.index)

• Compute Exponential Smoothing Forecasts

In [ ]:
# Proceed with model fitting
alphas = [0.1, 0.5, 0.8]
forecasts = {}
for alpha in alphas:
    forecasts[alpha] = exponential_smoothing(time_series, alpha)

# Plot the results
plt.figure(figsize=(12,6))
plt.plot(time_series.index, time_series.values, label='Actual Data', marker='o')

for alpha in alphas:
    plt.plot(forecasts[alpha].index, forecasts[alpha].values, label=f'Alpha = {alpha}', linestyle='--')
plt.title('Exponential Smoothing with Different Alphas')
plt.xlabel('Month')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()

• Save Forecasts to Spreadsheet

In [ ]:
# Add forecasted values to the DataFrame
for idx, alpha in enumerate(alphas, start=1):
    df_exp.iloc[idx, 1:] = forecasts[alpha].values
    df_exp.iloc[idx, 0] = f'Forecast Alpha={alpha}'
    
# Display the updated DataFrame
df_exp

• Save to File(Overwrite Existing File)

In [ ]:
with pd.ExcelWriter('/tmp/Assignment_3_DataSheet.xlsx', engine='openpyxl', 
                    mode='a', if_sheet_exists='replace') as writer:
    df_exp.to_excel(writer, sheet_name='Exponential Smoothing Results', index=False)

Scenario 2 - Moving Averages - Import Necessary Libraries & Load Data

In [ ]:
df_ma = pd.read_excel('/tmp/Assignment_3_DataSheet.xlsx', sheet_name='Moving␣Averages', header=2)

Prepare the Data

In [ ]:
# Get the periods from the columns, excluding the 'Bi-Weekly Period' column
periods = df_ma.columns[1:]
# Get the actual data from the first row, excluding the 'Bi-Weekly Period' column
actual_data = df_ma.iloc[0, 1:].values
# Create a pandas Series with periods as index
time_series = pd.Series(data=actual_data, index=periods.astype(int))

• Create the Line Graph of the Time Series

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(time_series.index, time_series.values, label='Original Data', marker='o')
plt.title('Athletes\' Performance Over Time')
plt.xlabel('Bi-Weekly Period')
plt.ylabel('Time')
plt.legend()
plt.grid(True)
plt.show()

• Calculate Moving Averages

In [ ]:
windows = [2, 4, 6]
ma_series = {}
for window in windows:
    ma = time_series.rolling(window=window).mean()
    ma_series[window] = ma

plt.figure(figsize=(12,6))
plt.plot(time_series.index, time_series.values, label='Original Data', marker='o')
for window in windows:
    plt.plot(ma_series[window].index, ma_series[window].values, label=f'MovingAverage (Window={window})')
plt.title('Moving Averages with Different Intervals')
plt.xlabel('Bi-Weekly Period')
plt.ylabel('Time')
plt.legend()
plt.grid(True)
plt.show()

• Save Moving Averages to Spreadsheet

In [ ]:
rows_list = []
# Original data
original_row = ['Original Data'] + list(time_series.values)
rows_list.append(original_row)
for window in windows:
    ma_values = ma_series[window].values
    row = [f'Moving Average Window={window}'] + list(ma_values)
    rows_list.append(row)
# Define the columns
columns = ['Bi-Weekly Period'] + list(time_series.index)
# Create the DataFrame
df_ma_results = pd.DataFrame(rows_list, columns=columns)
# Display the results
print(df_ma_results)

• Save File (Overwrite) with Moving Averages

In [ ]:
import pandas as pd
# Define the sheet name
sheet_name = 'Moving Averages Results'
# Use ExcelWriter with if_sheet_exists='replace' to overwrite the existing sheet
with pd.ExcelWriter('/tmp/Assignment_3_DataSheet.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_ma_results.to_excel(writer, sheet_name=sheet_name, index=False)

Scenario 3 – Forecasting - Load Data

In [ ]:
df_forecast = pd.read_excel('/tmp/Assignment_3_DataSheet.xlsx', sheet_name='Forecasting', header=2)
df_forecast['Month_Num'] = df_forecast['Month']
# Drop rows with NaN values (if any)
df_forecast = df_forecast.dropna(subset=['Boxes', 'Month_Num'])
# Ensure 'Boxes' is numeric
df_forecast['Boxes'] = pd.to_numeric(df_forecast['Boxes'])

In [ ]:
plt.figure(figsize=(12,6))
plt.scatter(df_forecast['Month_Num'], df_forecast['Boxes'], label='ActualData', color='blue')
plt.title('Number of Boxes Stored Over Time')
plt.xlabel('Month')
plt.ylabel('Number of Boxes')
plt.xticks(ticks=df_forecast['Month_Num'], labels=df_forecast['Month'])
plt.legend()
plt.grid(True)
plt.show()

• Add a Linear Trend Line

In [ ]:
from sklearn.linear_model import LinearRegression
# Prepare data
X = df_forecast['Month_Num'].values.reshape(-1, 1)
y = df_forecast['Boxes'].values
# Fit the model
model = LinearRegression()
model.fit(X, y)
# Predict values
y_pred = model.predict(X)

• Plot the trend line

In [ ]:
plt.figure(figsize=(12,6))
plt.scatter(df_forecast['Month_Num'], df_forecast['Boxes'], label='ActualData', color='blue')
plt.plot(df_forecast['Month_Num'], y_pred, label='Trend Line', color='red')
plt.title('Trend Line for Number of Boxes Stored')
plt.xlabel('Month')
plt.ylabel('Number of Boxes')
plt.xticks(ticks=df_forecast['Month_Num'], labels=df_forecast['Month'])
plt.legend()
plt.grid(True)
plt.show()

• Display the Forecast Equation

In [ ]:
slope = model.coef_[0]
intercept = model.intercept_
print(f'Forecast Equation: y = {slope:.2f} * x + {intercept:.2f}')

Forecast Equation: y = -50.86 * x + 1617.33
• Forecast the Next 2 Months

In [ ]:
# Add November and December to the dataset
future_months = ['Nov', 'Dec']
future_month_nums = [month_mapping[month] for month in future_months]
X_future = np.array(future_month_nums).reshape(-1, 1)
y_future_pred = model.predict(X_future)
# Create a DataFrame for future forecasts
df_future = pd.DataFrame({
'Month': future_months,
'Month_Num': future_month_nums,
'Forecast': y_future_pred
})
print(df_future)

• Plot the Forecasts on the Graph

In [ ]:
plt.figure(figsize=(12,6))
plt.scatter(df_forecast['Month_Num'], df_forecast['Boxes'], label='ActualData', color='blue')
plt.plot(df_forecast['Month_Num'], y_pred, label='Trend Line', color='red')
plt.scatter(df_future['Month_Num'], df_future['Forecast'], label='Forecasts', color='green', marker='X', s=100)
all_month_nums = df_forecast['Month_Num'].tolist() + df_future['Month_Num'].tolist()
all_months = df_forecast['Month'].tolist() + df_future['Month'].tolist()
plt.title('Forecasts for November and December')
plt.xlabel('Month')
plt.ylabel('Number of Boxes')
plt.xticks(ticks=all_month_nums, labels=all_months)
plt.legend()
plt.grid(True)
plt.show()

• Save the Forecasts to Spreadsheet

In [ ]:
# Append future forecasts to the DataFrame using pd.concat
df_combined = pd.concat([df_forecast, df_future], ignore_index=True)
# Update the 'Boxes' column with actual and forecasted values
df_combined['Boxes'] = df_combined['Boxes'].combine_first(df_combined['Forecast'])
# Display the updated DataFrame
df_combined[['Month', 'Boxes']]

• Save File

In [ ]:
with pd.ExcelWriter('/tmp/Assignment_3_DataSheet.xlsx', engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_combined[['Month', 'Boxes']].to_excel(writer, sheet_name='ForecastingResults', index=False)